# Glassdoor Salaries

## Salary Prediction Model

WIP

## Model setup

We need to import all the modules we'll be using from numpy, scipy, and keras:

In [1]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
from scipy.io import mmread
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
import math
import csv


import keras
import json
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
%pwd

u'/Users/phillip/Code/ai/fastai-courses/experiments'

Define Location of data directory and load training matrix into sparse matrix.


In [7]:
data_dir = '/Users/phillip/Data/salary-estimates'
sparse_training_matrix = mmread('%s/training-matrix-test.txt' % data_dir)
print "Done"

Done


- sparse matrix text = training row + " " + column (where value is +1)

Notes:

'''
a.todense() or a.M - Return a dense matrix representation of this matrix. (numpy.matrix)
a.A - Return a dense ndarray representation of this matrix. (numpy.array)
'''


In [8]:
print sparse_training_matrix.shape
print sparse_training_matrix.getrow(568370)
print coo_matrix(sparse_training_matrix.getrow(568370), dtype=np.bool)
print "Done"

(6516817, 1812571)
  (0, 0)	1.0
  (0, 0)	True
Done


In [9]:
def salaryToTarget(salary):
    return int(round((max((min((salary * 1.0, 600000.0)), 15000))-15000) / 5000, 0)) + 1

def targetToSalary(target):
    return ((target - 1) * 5000) + 15000

def logSalaryToTarget(logSalary):
    return salaryToTarget(math.pow(math.e, logSalary))

print salaryToTarget(15000)
print salaryToTarget(25000)
print salaryToTarget(2500000)
print targetToSalary(8)
print logSalaryToTarget(10.3089859934221)

salaries = np.zeros((6516817, 118), dtype=np.bool)

with open('%s/log-salaries-test-truncated.csv' % data_dir,'r') as dest_f:
    data_iter = csv.reader(dest_f)
    for row in data_iter:
        x_index = int(row[0])
        if x_index < 6516817:
            salaries[x_index][logSalaryToTarget(float(row[1]))] = True 
    
print "Created salaries"
    
#targets = mmread('%s/training-matrix.txt' % data_dir)

1
3
118
50000
4
Created salaries


## Model creation

### Features

- Binary vector of length 1812571 
- Each row is a 

### Training Data

- Log salary
- Salaries: $15K => $600K
- Divide into classes of $5K ranges => 117 classes

### Targets

- 118 length vector
- Note: 0 should be ignored


In [10]:
def ConvBlock(layers, model, filters):
    for i in range(layers): 
        model.add(ZeroPadding2D((1,1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

...and here's the fully-connected definition.

In [11]:
def FCBlock(model):
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))

Define the model

In [12]:
def PhillipSalary1():
    model = Sequential()
    
    model.add(Dense(512, input_shape=(1812571,)))
    # todo this is definitely not correct
    #ConvBlock(3, model, 128)
    #model.add(Flatten())
    #FCBlock(model)
    #FCBlock(model)
    model.add(Dense(118, activation='softmax'))
    return model

We'll learn about what these different blocks do later in the course. For now, it's enough to know that:

- Convolution layers are for finding patterns in images
- Dense (fully connected) layers are for combining patterns across an image

Now that we've defined the architecture, we can create the model like any python object:

In [13]:
model = PhillipSalary1()

As well as the architecture, we need the weights that the VGG creators trained. The weights are the part of the model that is learnt from the data, whereas the architecture is pre-defined based on the nature of the problem. 

Downloading pre-trained weights is much preferred to training the model ourselves, since otherwise we would have to download the entire Imagenet archive, and train the model for many days! It's very helpful when researchers release their weights, as they did here.

In [14]:
lr = 0.01

In [39]:
### TESTING: Going to create a new training matrix with 10 rows

test_y = salaries[:100]

print "Finished Salary Matrix: "

print test_y

#test_x = [] 



csr_sparse_training_matrix = csr_matrix(sparse_training_matrix, dtype=np.bool)
test_x = csr_sparse_training_matrix[:100]

print "Finished CSR Training Matrix"

print test_x

#test_x.shape
#for n in range(10)
print "Training Set: %s" % len(test_x.A)

print "Features: %s" % len(test_x.A[0])

#y = np.zeros(118, dtype=np.int32)
#y[18] = 1
#y

Finished Salary Matrix: 
[[False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]
 ..., 
 [False False False ..., False False False]
 [False False False ..., False False False]
 [False False False ..., False False False]]
Finished CSR Training Matrix
  (9, 12)	True
  (11, 12)	True
  (16, 12)	True
  (18, 12)	True
  (19, 12)	True
  (39, 198)	True
  (49, 198)	True
  (54, 77)	True
  (55, 168)	True
  (64, 168)	True
  (65, 77)	True
  (66, 145)	True
  (67, 168)	True
  (68, 145)	True
  (72, 77)	True
  (80, 77)	True
  (82, 76)	True
  (86, 77)	True
  (89, 168)	True
  (93, 77)	True
Training Set: 100
Features: 1812571


In [107]:
model.compile(optimizer=SGD(lr=lr), loss='categorical_crossentropy', metrics=['accuracy'])

In [108]:
history = model.fit(x=sparse_training_matrix, y=salaries, shuffle="batch")

Epoch 1/10


TypeError: TypeError while preparing batch. If using HDF5 input data, pass shuffle="batch".